# Submit CP2K DFT Calculation

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)

from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.structure import StructureData
from aiida.orm.data.parameter import ParameterData
from aiida.orm.code import Code
from aiida.orm.computer import Computer
from aiida.orm import load_node 

import ipywidgets as ipw
from IPython.display import display, clear_output, HTML
import nglview

from collections import OrderedDict
width = "150px"

## Step 1: Select Structure from Database

In [ ]:
# query AiiDA database for structures
qb = QueryBuilder()
qb.append(StructureData, project=["id", "description"])
qb.order_by({StructureData: {'ctime': 'desc'}})
#print qb.count()

all_structs = OrderedDict()
all_structs["Select a Structure"] = False
for pk, descr in qb.iterall():
    label = "PK: %d"%pk
    if descr:
        label += "; descr: "+descr
    all_structs[label] = pk

structure = None
def on_struct_change(c):
    global structure, atoms
    while hasattr(viewer, "component_0"):
        viewer.component_0.remove_ball_and_stick()
        viewer.component_0.remove_unitcell()
        cid = viewer.component_0.id
        viewer.remove_component(cid)
    if drop_struct.value:
        structure = load_node(drop_struct.value)
        atoms = structure.get_ase()
        viewer.add_component(nglview.ASEStructure(atoms)) # adds ball+stick
        viewer.add_unitcell()
        viewer.center_view()

drop_struct = ipw.Dropdown(options=all_structs, value=False)
drop_struct.observe(on_struct_change, names='value')
viewer = nglview.NGLWidget()
display(ipw.VBox([drop_struct, viewer]))

## Step 2: Choose parameters

In [ ]:
def get_computer_names():
    #https://github.com/aiidateam/aiida_core/blob/develop/aiida/cmdline/commands/computer.py#L1047
    qb = QueryBuilder()
    qb.append(Computer, project=['name'])
    qb.order_by({Computer: {'id': 'desc'}})
    return zip(*qb.all())[0]

In [ ]:
rows = []
w = "100px"

inp_compname = ipw.Dropdown(options=get_computer_names())
rows.append(ipw.HBox([ipw.Label("Computer", width=width), inp_compname]))

inp_xc = ipw.Dropdown(options=["LDA", "PBE"])
rows.append(ipw.HBox([ipw.Label("XC Functional", width=width), inp_xc]))

inp_basis = ipw.Dropdown(options=["SZV-MOLOPT-SR-GTH", "DZVP-MOLOPT-SR-GTH"])
rows.append(ipw.HBox([ipw.Label("Basis Set", width=width), inp_basis]))

inp_descr = ipw.Text(placeholder="(optional)")
rows.append(ipw.HBox([ipw.Label("Description", width=width), inp_descr]))

display(ipw.VBox(rows))

## Step 3: Choose properties

In [ ]:
rows = []

options = OrderedDict()
options['none'] = None
options['gap +- 5'] = 5
options['gap +- 10'] = 10
options['all'] = -1
inp_mos = ipw.Dropdown(options=options)
rows.append(ipw.HBox([ipw.Label("Molecular Orbitals", width=width), inp_mos]))

display(ipw.VBox(rows))

In [ ]:
def build_parameters():
    kinds = []
    for k in structure.get_kind_names():
        kinds.append({'_':k, 'BASIS_SET': inp_basis.value,  'POTENTIAL': "GTH-"+inp_xc.value})

    print_sec = {}
    if inp_mos.value:
        print_sec['MO_CUBES'] = {'NHOMO': inp_mos.value, 'NLUMO': inp_mos.value}
    
    force_eval = {
            'METHOD': 'Quickstep',
            'DFT': {
                'BASIS_SET_FILE_NAME': 'BASIS_MOLOPT',
                'XC': {
                    'XC_FUNCTIONAL': {
                        '_': inp_xc.value,
                    },
                },
                'PRINT': print_sec,
                
            },
            'SUBSYS': {
                'KIND': kinds,
            },
        }
    return ParameterData(dict={'FORCE_EVAL':force_eval, 'GLOBAL':{'RUN_TYPE':'energy'}})

In [ ]:
def build_calc():
    code = Code.get_from_string("cp2k@"+inp_compname.value)

    # calc object
    calc = code.new_calc()
    calc.description = inp_descr.value

    # structure
    calc.use_structure(structure)

    # parameters
    parameters = build_parameters()
    calc.use_parameters(parameters)

    # settings
    if inp_mos.value:
        settings = ParameterData(dict={'additional_retrieve_list':['*.cube']})
        calc.use_settings(settings)
    
    # resources
    calc.set_max_wallclock_seconds(3*60) # 3 min
    calc.set_resources({"num_machines": 1})
    return calc

## Step 4: Test submission

In [ ]:
def on_test(b):
    with test_out:
        clear_output()
        if structure is None:
            print("Please select a structure.")
            return
        calc = build_calc()
        folder = calc.submit_test()[0]
        fn = folder.get_abs_path(calc._INPUT_FILE_NAME)
        content = open(fn).read()
        pre_tag = '<pre style="width:600px; max-height:250px; overflow-x:auto; line-height:1em; font-size:0.8em;">'   
        box_preview.value = pre_tag+content+"</pre>"
        # toggle visibility to workaround an apparent bug
        #box_preview.visible = not chk_preview.value
        #box_preview.visible = chk_preview.value
        print("Test went fine :-)")

btn_test = ipw.Button(description="Test")
btn_test.on_click(on_test)
#chk_preview = ipw.Checkbox(description='preview', value=False)
box_preview = ipw.HTML()
#ipw.jslink((chk_preview, 'value'), (box_preview, 'visible'))
#display(ipw.VBox([ipw.HBox([btn_test, chk_preview]), box_preview]))
#box_preview.layout.visibility
test_out = ipw.Output()
display(ipw.VBox([btn_test, box_preview]), test_out)

## Step 5: Submit calculation

In [ ]:
def on_submit(b):
    with submit_out:
        clear_output()
        calc = build_calc()
        # store and submit
        calc.store_all()
        calc.submit()
        msg = "Submitted calculation: UUID=%s, pk=%s .<br>"%(calc.uuid,calc.dbnode.pk)
        msg += 'Results can be viewed <a href="view_cp2k_calc.ipynb?pk=%d">here</a>'%calc.dbnode.pk
        display(HTML(msg))

btn_submit = ipw.Button(description="Submit")
btn_submit.on_click(on_submit)
submit_out = ipw.Output()
display(btn_submit, submit_out)